# Cycle GAN

Based on paper [Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks](https://arxiv.org/abs/1703.10593)

In [10]:
import torch
from torch import nn
import math

In [1]:
from models import generative,discriminative

In [ ]:
G_1 = generative([3,128,128,256,128])
D_1 = discriminative([64,64,64,128,128,256,256])

G_2 = generative([3,128,128,256,128]) 
D_2 = discriminative([64,64,64,128,128,256,256])

In [123]:
D = discriminative([64,64,64,128,128,256,256])

In [125]:
D

discriminative(
  (conv_in): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_in): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res_0): resblock_d(
    (leaky): LeakyReLU(negative_slope=0.01)
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (out): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn_out): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res_1): resblock_d(
    (leaky): LeakyReLU(negative_slope=0.01)
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn_1): BatchNorm2d(6

In [124]:
D(torch.rand(2,3,256,256)).size()

torch.Size([2, 256, 4, 4])

In [97]:
G = generative([3,128,128,128,256,256])

In [98]:
G

generative(
  (leaky): LeakyReLU(negative_slope=0.01)
  (tran_0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_trans_0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (resblock_0): resblock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky1): LeakyReLU(negative_slope=0.01)
    (upconv): Upsample(scale_factor=2, mode=nearest)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (bn_res_0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (tran_1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_trans_1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (resblock_1): resblock(
    (conv1): Conv2d(128, 128, kernel_size=(3

In [96]:
G(torch.rand(2,3,160,160)).size()

torch.Size([2, 3, 160, 160])